In [1]:
import pandas as pd
import numpy as np
import re
import itertools

# What do I do?

I import the data, I group it by District and Verkiezingsdatum (so as to have unique groups), and then I calculate positions. 

Then I calculate the winning and losing candidates by taking "if <position> > <Aantal zetels>" then Loser, otherwise winner. 
    
Then, I calculate the margins for the losers to the lowest winner. 
    
Finally, I make a new dataset containing only the closely elected politicians and the names of the winners of the district.

In [2]:
data = pd.read_csv("../Data/elections/election_results_details.csv", encoding='latin-1')
data = data.drop('Unnamed: 0', axis=1)

## First, make a counter

In [3]:
def counter(group):
    temp = pd.Series(range(0,len(group)), dtype='int')
    temp = temp + 1
    return temp

In [4]:
data["counts"] = data.groupby(['District', 'Verkiezingdatum'], sort = False)['Naam'].transform(counter)

## Then, make win indicators

In [5]:
data['Aantal zetels'] = pd.to_numeric(data['Aantal zetels'], errors='coerce')

data["won"] = np.where(data["counts"] <= data['Aantal zetels'], 1,0)

## Now, calculate the margin to the lowest winner

In [6]:
winners = data.groupby(['District', 'Verkiezingdatum'], sort = False, as_index = False).apply(lambda x: x[x['won'] == 1])
winners['Aantal stemmen'] = pd.to_numeric(winners['Aantal stemmen'])

In [7]:
minwinners = winners.loc[winners.groupby(['District', 'Verkiezingdatum'], sort = False, as_index = False)['Aantal stemmen'].idxmin()]
minwinners = minwinners.reset_index().iloc[:,2:]

# Now

Make a per unique election min_procentueel and min_aantal_stemmen

Then merge them with the large dataframe with all candidates

And calculate the margn

If negative, then they lost. We then proceed to check a couple of things

In [8]:
#Make a per unique election min_procentueel and min_aantal_stemmen
minwinners = minwinners.iloc[:,[4,5,2,3]]

minwinners = minwinners.rename({'Aantal stemmen':'min_stem_win', 
                  'Procentueel':'min_pct_win'}, axis=1)

In [9]:
#Then merge them with the large dataframe with all candidates
allmgn = data.merge(minwinners, how='left', left_on=['District','Verkiezingdatum'], right_on=['District','Verkiezingdatum'])

In [10]:
#Calculate the margin
allmgn['Aantal stemmen'] = pd.to_numeric(allmgn['Aantal stemmen'], errors='coerce')

def p2f(x):
    var = []
    for i in x:
        var.append(pd.to_numeric(i.strip('%'), errors='coerce')/100)
    return var

allmgn['Procentueel'] = p2f(allmgn['Procentueel'])

In [11]:
def p2f_nan(x):
    var = []
    for i in x:
        if type(i) == str:
            var.append(pd.to_numeric(i.strip('%'), errors='coerce')/100)
        else:
            var.append(i)
    return var

allmgn['min_pct_win'] = p2f_nan(allmgn['min_pct_win'])

In [12]:
allmgn['margin'] = allmgn['Procentueel'] - allmgn['min_pct_win']

In [13]:
# If negative, they lost:

def close_cand(x, pct):
    var = []
    for i in x:
        var.append(pd.to_numeric(np.where(i < 0 and i > -pct, 1, 0)))
    return pd.Series(var)
        

allmgn['close_cand_5pct'] = close_cand(allmgn['margin'], 0.05)
allmgn['close_cand_10pct'] = close_cand(allmgn['margin'], 0.10)
allmgn['close_cand_20pct'] = close_cand(allmgn['margin'], 0.20)

In [17]:
allmgn['Kiesdrempel'] = pd.to_numeric(allmgn['Kiesdrempel'], errors='coerce')
allmgn['Aantal stemmen geldig'] = pd.to_numeric(allmgn['Aantal stemmen geldig'], errors='coerce')

#allmgn[allmgn['Aantal stemmen geldig'] > allmgn['Kiesdrempel']]
#Everything is larger than kiesdrempel

## Finally..

Take the data frame, take only the close losers

- For each close loser, look up the (just) winner in that election (only 1 person)
- Match each loser to that winner (multiple matches per district possible if e.g. 2 close losing candidates)
    - Merge strategy: take all the eligible just losers
    - Merge them to a filtered dataset finished with the just winners on the basis of district, date (left join with left = just losers)



In [171]:
# All just Losers
justlosers = allmgn[allmgn['close_cand_20pct'] == 1]


# Filtered dataset with just winners
justwinners = allmgn[allmgn['counts'] == allmgn['Aantal zetels']]

# Filtered dataset with all winners
allwinners = allmgn[allmgn['counts'] <= allmgn['Aantal zetels']]

In [172]:
#change the varnames for the justwinners to avoid duplicate issues
justwinners = justwinners.reset_index().iloc[:,1:7].rename({
    'Naam':'Naam_win',
    'Aanbevolen door':'Aanbevolen door_win',
    'Aantal stemmen':'Aantal stemmen_win',
    'Procentueel':'Procentueel_win'}, axis=1)

In [173]:
# Merge justwinners with just losers
closepairs = justlosers.merge(justwinners, how='left', on=['District','Verkiezingdatum'])

# Retrieve some additional information

- Are they on parlement.com?
- Sterfplaats
- Sterfteleeftijd (geboortejaar)
- Distance to Den Haag via Google Maps API

In [182]:
#J'ai utilise ce fichier auparavant pour générer data_entry.csv sur Google Drive
#Maintenant j'en fais un fichier plus précis, quitte à omettre quelques matches potentiels 

#closepairs.to_csv("../Administration/close_pairs.csv")

politici = allwinners['Naam'].drop_duplicates()
#politici = politici.drop_duplicates()

closepairs['alrdy_in_pols'] = np.where(closepairs['Naam'].isin(politici),'ja','nee')

closepairs2 = closepairs[closepairs['alrdy_in_pols'] == 'nee']

In [190]:
closepairs2.to_csv("../Administration/close_pairs2.csv")

## Make a new sheet with non-close pairs to enlarge the dataset


(24-03-2021): The dataset of close elections is almost complete. Hence, we enlarge the scope of the dataset to more (non-close or less close) elections. This is implemented in the following code. We then write a cv file to the Administration folder, nonclose_pairs, and use another .py script to make that into an easily-accessible data-entry file. 

In [28]:
allmgn['close_cand_30pct'] = close_cand(allmgn['margin'], 0.30)

allmgn[(allmgn['close_cand_20pct'] == 0) & (allmgn['close_cand_30pct'] == 1)]

#From this set, make sure only to have the losers, not the politicians
#Also, make sure to have no overlap with the entry_file dataset in names (import it here, and filter on that basis)

,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,...,Kiesdrempel,counts,won,min_stem_win,min_pct_win,margin,close_cand_5pct,close_cand_10pct,close_cand_20pct,close_cand_30pct
6,mr. A. Oudeman,NaN,113.0,0.1339,Onderdendam,30/11/1848,algemeen,1568,847,844,...,422.0,2,0,357.0,0.4230,-0.2891,0,0,0,1
39,dr. P.J. Costerus,NaN,155.0,0.1707,Sneek,30/11/1848,algemeen,1168,907,908,...,454.0,2,0,366.0,0.4031,-0.2324,0,0,0,1
40,mr. A.F. Jongstra,NaN,139.0,0.1531,Sneek,30/11/1848,algemeen,1168,907,908,...,454.0,3,0,366.0,0.4031,-0.2500,0,0,0,1
52,mr. G. Groen van Prinsterer,NaN,129.0,0.1794,Franeker,19/12/1848,vacature,1158,725,725,...,360.0,3,0,329.0,0.4576,-0.2782,0,0,0,1
59,mr. G. Kniphorst,NaN,47.0,0.0568,Assen,30/11/1848,algemeen,973,835,835,...,414.0,4,0,245.0,0.2959,-0.2391,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8034,S.W. de Clercq,NaN,3993.0,0.3805,Haarlemmermeer,02/12/1916,tussentijds,14679,-,10595,...,5247.0,2,0,6500.0,0.6195,-0.2390,0,0,0,1
8043,J.A. Bergmeyer,NaN,1910.0,0.3728,Emmen,28/02/1917,tussentijds,11929,-,5191,...,2562.0,2,0,3213.0,0.6272,-0.2544,0,0,0,1
8129,dr. J. Schrijver,AG,2448.0,0.3444,Amsterdam V,15/06/1917,algemeen,20627,-,7278,...,3554.0,2,0,4077.0,0.5736,-0.2292,0,0,0,1
8136,dr. F. van Eeden,AS,828.0,0.3292,Amsterdam VIII,15/06/1917,algemeen,7444,-,2638,...,1258.0,2,0,1526.0,0.6068,-0.2776,0,0,0,1
